In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pgf import FigureCanvasPgf
from matplotlib.ticker import MaxNLocator
import matplotlib.gridspec as gridspec
import matplotlib.ticker as ticker
import os

In [ ]:
def import_data(filename):
    zeros = []
    random = []
    with open(filename, 'r') as file:
        for line in file:
            line = line.split(' ')

            zeros.append(int(line[0]))
            random.append(int(line[1]))
    return zeros, random

In [ ]:
def remove_outliers(data):
    d = []
    for i, v in enumerate(data):
        d.append((i, v))

    d = sorted(d, key=lambda x: x[1])

    d = d[(len(d) // 200):-(len(d) // 200)]

    d = sorted(d, key=lambda x: x[0])
    return [x[1] for x in d]

In [ ]:
path = "../ct_results_M3_dit"
filenames = [
    "ntru:hps2048509:stack:NG21:amx.txt",
    "ntru:hps2048509:mmap:NG21:amx.txt",
    "ntru:hps2048509:mmap:NG21:neon.txt",
]

zeros = [None]*3
random = [None]*3

for i, filename in enumerate(filenames):
    zeros[i], random[i] = import_data(os.path.join(path, filename))
    zeros[i] = remove_outliers(zeros[i])
    random[i] = remove_outliers(random[i])

In [ ]:
def plot_histogram_heat_maps(zeros, random, title):
    n = 1000
    fig = plt.figure(figsize=(15, 3))
    fig.subplots_adjust(wspace=0.1)

    gs = gridspec.GridSpec(1, 4, width_ratios=[1, 1, 1, .03])

    d = {"zeros": zeros, "random": random}
    ax_map = {"zeros": 1, "random": 2}
    hist = {}
    axs = []
    for i in range(4):
        axs.append(fig.add_subplot(gs[0, i]))

    ymin = min(min(d["zeros"]), min(d["random"]))
    ymax = max(max(d["zeros"]), max(d["random"]))

    for input in ["zeros", "random"]:
        d_in = d[input]
        color = "black" if input == "zeros" else "silver"

        axs[0].hist(d_in, bins=range(ymin, ymax + 1), alpha=0.75, color=color, label=input, orientation='horizontal')

        ax = axs[ax_map[input]]
        hist[input] = ax.hist2d(
            range(len(d_in)),
            d_in,
            bins=[range(0, len(d_in), n), range(ymin, ymax + 1)],
            cmap="binary",
            alpha=0.75,
            label=input,
        )
        ax.set_xlabel('Sample index (x1000)')
        ax.set_ylim(ymin, ymax)
        ax.set_title(input)
        ax.yaxis.set_tick_params(labelleft=False)

    fig.colorbar(hist["random"][3], cax=axs[3], format=ticker.FuncFormatter(lambda x, pos : f"{x/n:.2f}"))

    axs[0].yaxis.set_major_locator(MaxNLocator(integer=True))
    axs[0].set_ylabel('Cycles')
    axs[0].set_xlabel('Samples (x1000)')
    axs[0].set_ylim(ymin, ymax)
    axs[0].invert_xaxis()
    axs[0].set_title(title)
    axs[0].legend()

    for i in range(3):
        axs[i].xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos : f"{x/1000:.0f}"))


In [ ]:
plot_histogram_heat_maps(zeros[0], random[0], "HPS2048509, stack, ours")

In [ ]:
plot_histogram_heat_maps(zeros[1], random[1], "HPS4096821, mmap, ours")

In [ ]:
plot_histogram_heat_maps(zeros[2], random[2], "HRSS701, stack, TMVP")

In [ ]:
def plot_3_histogram_heat_maps(zeros, random):
    n = 1000
    fig = plt.figure(figsize=(10, 6), dpi=600)
    fig.subplots_adjust(wspace=0.1, hspace=0.2)

    gs = gridspec.GridSpec(3, 4, width_ratios=[1, 1, 1, .03])
    labels = ["(a)", "(b)", "(c)"]

    for i in range(3):
        d = {"zeros": zeros[i], "random": random[i]}
        ax_map = {"zeros": 1, "random": 2}
        hist = {}
        axs = []
        for j in range(4):
            axs.append(fig.add_subplot(gs[i, j]))

        ymin = min(min(d["zeros"]), min(d["random"]))
        ymax = max(max(d["zeros"]), max(d["random"]))

        for input in ["zeros", "random"]:
            d_in = d[input]
            color = "black" if input == "zeros" else "silver"

            axs[0].hist(d_in, bins=range(ymin, ymax + 1), alpha=0.75, color=color, label=input, orientation='horizontal')

            ax = axs[ax_map[input]]
            hist[input] = ax.hist2d(
                range(len(d_in)),
                d_in,
                bins=[range(0, len(d_in), n), range(ymin, ymax + 1)],
                cmap="binary",
                alpha=0.75,
                label=input,
                rasterized=True,
            )
            if i == 2:
                ax.set_xlabel('Sample index ($\\times 1000$)')
            ax.set_ylim(ymin, ymax)
            if i == 0:
                ax.set_title(input)
            ax.yaxis.set_tick_params(labelleft=False)

        fig.colorbar(hist["random"][3], cax=axs[3], format=ticker.FuncFormatter(lambda x, pos : f"{x/n:.2f}"))

        axs[0].yaxis.set_major_locator(MaxNLocator(integer=True))
        axs[0].set_ylabel('Cycles')
        if i == 2:
            axs[0].set_xlabel('Samples ($\\times 1000$)')
        # axs[0].set_xlim(0, 165e3)
        axs[0].set_ylim(ymin, ymax)
        axs[0].invert_xaxis()
        axs[0].legend()

        for j in range(3):
            axs[j].xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos : f"{x/1000:.0f}"))
            # if i < 2:
            #     axs[j].xaxis.set_ticklabels([])
    
        fig.text(0.03, 0.76 - 0.265*i, labels[i])

    plt.savefig(os.path.join(path, "histograms_heat_maps", "histograms_heat_maps.pgf"), bbox_inches='tight', pad_inches=0)

In [ ]:
matplotlib.use("pgf")
matplotlib.backend_bases.register_backend('pgf', FigureCanvasPgf)
plt.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
    'path.simplify': True,
})

In [ ]:
plot_3_histogram_heat_maps(zeros, random)